In [32]:
import json
import os
import sys
from collections import defaultdict
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd

# Paths and inputs
repo = Path('..').resolve()
coverage_path = repo / 'coverage_table_updated.csv'
validation_path = repo / 'validation_results.csv'
candidates_path = repo / 'report_candidates.csv'

coverage = pd.read_csv(coverage_path, sep='\t')
candidates = pd.read_csv(candidates_path)

# Load validation results to skip already-validated reports
validated_df = pd.read_csv(validation_path)
validated_set = set(
    (row['CID'], row['Year']) 
    for _, row in validated_df[validated_df['Valid'] == True].iterrows()
)
print(f"✓ Already validated: {len(validated_set)} company-year combinations")

# Filter to incomplete rows that are NOT already validated
incomplete = coverage[coverage['Priority'] != 'Complete ✓'].copy()

def needs_download(row):
    return (row['Company_Identifier'], row['FiscalYear']) not in validated_set

incomplete = incomplete[incomplete.apply(needs_download, axis=1)]
incomplete = incomplete[['CompanyName', 'Company_Identifier', 'FiscalYear', 'IR_URL']]

print(f"→ Reports to download: {len(incomplete)}")
print(f"→ Candidates available: {len(candidates)}")

# Show summary by company
by_company = incomplete.groupby('CompanyName').size().sort_values(ascending=False)
print(f"\nCompanies needing downloads ({len(by_company)}):")
for company, count in by_company.head(10).items():
    print(f"  {company}: {count} years")

✓ Already validated: 0 company-year combinations
→ Reports to download: 180
→ Candidates available: 176

Companies needing downloads (30):
  ABB Ltd: 6 years
  ASSA ABLOY B: 6 years
  Telia Company: 6 years
  Tele2 B: 6 years
  Swedbank A: 6 years
  Sv. Handelsbanken A: 6 years
  Skanska B: 6 years
  Sandvik: 6 years
  SKF B: 6 years
  SEB A: 6 years


In [33]:
# Download PDFs using precomputed candidates from step 3
from aspiratio.utils.report_downloader import download_pdf

def download_company_year(cid, company, year, ir_url, company_candidates):
    """Download a single company-year report, trying all candidates."""
    result = {
        'cid': cid,
        'company': company,
        'year': year,
        'status': 'failed',
        'url': '',
        'source_page': ir_url,
        'error': '',
        'pages': 0,
        'size_mb': 0.0
    }
    
    if company_candidates.empty:
        result['error'] = 'No candidate URL for year'
        return result
    
    # Try each candidate until one succeeds
    for _, cand in company_candidates.iterrows():
        url = cand['URL']
        title = cand.get('Title', '')
        source_page = cand.get('Source_Page', ir_url) or ir_url
        output_path = repo / 'companies' / cid / f"{cid}_{year}_Annual_Report.pdf"
        
        # Create directory if needed
        output_path.parent.mkdir(parents=True, exist_ok=True)
        
        dl_result = download_pdf(url, str(output_path), year_hint=year)
        
        if dl_result.get('success'):
            result['status'] = 'success'
            result['url'] = url
            result['source_page'] = source_page
            result['title'] = title
            result['pages'] = dl_result.get('pages', 0)
            result['size_mb'] = dl_result.get('size_mb', 0.0)
            result['output_path'] = str(output_path)
            return result
        else:
            result['error'] = dl_result.get('error', 'download failed')
            result['url'] = url
    
    return result

# Build download tasks
download_tasks = []
for _, row in incomplete.iterrows():
    cid = row['Company_Identifier']
    company = row['CompanyName']
    year = int(row['FiscalYear'])
    ir_url = row['IR_URL']
    company_candidates = candidates[(candidates['Company_Identifier'] == cid) & (candidates['Year'] == year)]
    download_tasks.append((cid, company, year, ir_url, company_candidates))

print(f"📥 Starting downloads for {len(download_tasks)} reports...")

# Download in parallel (but limit to 3 concurrent to be respectful to servers)
download_results = []
max_workers = min(3, len(download_tasks)) if download_tasks else 1

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = {
        executor.submit(download_company_year, cid, company, year, ir_url, cands): (cid, company, year)
        for cid, company, year, ir_url, cands in download_tasks
    }
    
    for future in as_completed(futures):
        cid, company, year = futures[future]
        try:
            result = future.result()
            download_results.append(result)
            if result['status'] == 'success':
                print(f"  ✓ {company} ({cid}) {year}: {result['pages']} pages")
            else:
                print(f"  ✗ {company} ({cid}) {year}: {result['error']}")
        except Exception as e:
            print(f"  ✗ {company} ({cid}) {year}: Exception - {e}")
            download_results.append({
                'cid': cid, 'company': company, 'year': year,
                'status': 'failed', 'error': str(e)
            })

# Convert to DataFrame for easier analysis
results_df = pd.DataFrame(download_results)

# Persist summary
download_summary_path = repo / 'download_summary_from_candidates.json'
with open(download_summary_path, 'w') as f:
    json.dump(download_results, f, indent=2, default=str)

print(f"\n📊 Download summary saved to: {download_summary_path}")
results_df.head(10)

📥 Starting downloads for 180 reports...
  → Downloading from: https://mb.cision.com/Main/417/3045869/1201088.pdf
  → Downloading from: https://mb.cision.com/Main/417/3296284/1379492.pdf
  → Downloading from: https://mb.cision.com/Main/417/3516666/1542750.pdf
  → Saving to: /Users/jakobjohannesson/Documents/github_repos/aspiratio/companies/S1/S1_2021_Annual_Report.pdf
  → Saving to: /Users/jakobjohannesson/Documents/github_repos/aspiratio/companies/S1/S1_2019_Annual_Report.pdf
  → Saving to: /Users/jakobjohannesson/Documents/github_repos/aspiratio/companies/S1/S1_2020_Annual_Report.pdf
  → Downloaded: 0.0 MB
  → Validating PDF...
  → PDF has 1 pages
  ✗ Only 1 pages (min 50 required)
  → Downloaded: 0.1 MB
  → Validating PDF...
  → Downloading from: https://mb.cision.com/Main/417/3725044/1882197.pdf
  → PDF has 1 pages
  ✗ Only 1 pages (min 50 required)
  ✗ ABB Ltd (S1) 2019: Only 1 pages (min 50 required)
  ✗ ABB Ltd (S1) 2021: Only 1 pages (min 50 required)
  → Downloading from: https

ignore '/Perms' verify failed


  → PDF has 386 pages
  ✓ Success: 386 pages, 9.2 MB
  ✓ Nordea Bank Abp (S19) 2023: 386 pages
  → Downloading from: https://mb.cision.com/Main/183/3052123/1205536.pdf
  → Downloaded: 10.8 MB
  → Validating PDF...
  → PDF has 292 pages
  ✓ Success: 292 pages, 10.8 MB
  ✓ Nordea Bank Abp (S19) 2020: 292 pages
  → Downloading from: https://mb.cision.com/Main/183/3300345/1382470.pdf
  → Saving to: /Users/jakobjohannesson/Documents/github_repos/aspiratio/companies/S20/S20_2019_Annual_Report.pdf
  → Downloaded: 0.0 MB
  → Validating PDF...
  → PDF has 1 pages
  ✗ Only 1 pages (min 50 required)
  → Downloading from: https://mb.cision.com/Public/183/3052123/97656ad42e7c7ce7.pdf
  → Saving to: /Users/jakobjohannesson/Documents/github_repos/aspiratio/companies/S20/S20_2020_Annual_Report.pdf
  → Downloading from: https://www.nordea.com/en/doc/annual-report-nordea-bank-abp-2024-0.pdf#page=5
  → Saving to: /Users/jakobjohannesson/Documents/github_repos/aspiratio/companies/S20/S20_2019_Annual_Repor

,cid,company,year,status,url,source_page,error,pages,size_mb,title,output_path
0,S1,ABB Ltd,2019,failed,https://mb.cision.com/Main/417/3045869/1201088...,https://global.abb/group/en/investors,Only 1 pages (min 50 required),0,0.000000,NaN,NaN
1,S1,ABB Ltd,2021,failed,https://mb.cision.com/Main/417/3516666/1542750...,https://global.abb/group/en/investors,Only 1 pages (min 50 required),0,0.000000,NaN,NaN
2,S1,ABB Ltd,2020,failed,https://mb.cision.com/Main/417/3296284/1379492...,https://global.abb/group/en/investors,Only 1 pages (min 50 required),0,0.000000,NaN,NaN
3,S1,ABB Ltd,2022,failed,https://mb.cision.com/Main/417/3725044/1882197...,https://global.abb/group/en/investors,Only 1 pages (min 50 required),0,0.000000,NaN,NaN
4,S1,ABB Ltd,2023,failed,https://global.abb/content/dam/abb/global/grou...,https://global.abb/group/en/investors,Only 23 pages (min 50 required),0,0.000000,NaN,NaN
5,S1,ABB Ltd,2024,failed,https://storage.mfn.se/c/aHR0cHM6Ly9lcXMtY29ja...,https://global.abb/group/en/investors,Only 2 pages (min 50 required),0,0.000000,NaN,NaN
6,S2,Addtech B,2019,success,https://www.addtech.com/fileadmin/user_upload/...,https://www.addtech.com/investors-and-media/an...,,61,11.168268,pdf,/Users/jakobjohannesson/Documents/github_repos...
7,S2,Addtech B,2020,success,https://www.addtech.com/fileadmin/user_upload/...,https://www.addtech.com/investors-and-media/an...,,74,13.500546,pdf,/Users/jakobjohannesson/Documents/github_repos...
8,S2,Addtech B,2021,success,https://www.addtech.com/fileadmin/user_upload/...,https://www.addtech.com/investors-and-media/an...,,74,12.474593,pdf,/Users/jakobjohannesson/Documents/github_repos...
9,S2,Addtech B,2022,success,https://www.addtech.com/fileadmin/user_upload/...,https://www.addtech.com/investors-and-media/an...,,80,14.207037,pdf,/Users/jakobjohannesson/Documents/github_repos...


In [34]:
# Summarize download outcomes
successes = results_df[results_df['status'] == 'success']
failures = results_df[results_df['status'] == 'failed']

print('Download Summary')
print('=' * 50)
print(f"✓ Successful downloads: {len(successes)}")
print(f"✗ Failed downloads: {len(failures)}")
print(f"Total attempted: {len(results_df)}")

if len(successes) > 0:
    print(f"\nSuccessful downloads:")
    for _, row in successes.iterrows():
        pages = row.get('pages', '?')
        print(f"  ✓ {row['company']} ({row['cid']}) {row['year']}: {pages} pages")

if len(failures) > 0:
    print(f"\nFailed downloads (by error type):")
    error_counts = failures.groupby('error').size().sort_values(ascending=False)
    for error, count in error_counts.items():
        print(f"  • {error}: {count}")
    
    print(f"\nFailed details:")
    for _, row in failures.iterrows():
        print(f"  ✗ {row['company']} ({row['cid']}) {row['year']}: {row['error']}")

Download Summary
✓ Successful downloads: 135
✗ Failed downloads: 45
Total attempted: 180

Successful downloads:
  ✓ Addtech B (S2) 2019: 61 pages
  ✓ Addtech B (S2) 2020: 74 pages
  ✓ Addtech B (S2) 2021: 74 pages
  ✓ Addtech B (S2) 2022: 80 pages
  ✓ Addtech B (S2) 2024: 152 pages
  ✓ Addtech B (S2) 2023: 155 pages
  ✓ Alfa Laval (S3) 2019: 152 pages
  ✓ Alfa Laval (S3) 2020: 160 pages
  ✓ Alfa Laval (S3) 2021: 194 pages
  ✓ Alfa Laval (S3) 2023: 192 pages
  ✓ Alfa Laval (S3) 2024: 81 pages
  ✓ Alfa Laval (S3) 2022: 200 pages
  ✓ ASSA ABLOY B (S4) 2019: 108 pages
  ✓ ASSA ABLOY B (S4) 2020: 112 pages
  ✓ ASSA ABLOY B (S4) 2022: 116 pages
  ✓ ASSA ABLOY B (S4) 2021: 116 pages
  ✓ ASSA ABLOY B (S4) 2023: 116 pages
  ✓ ASSA ABLOY B (S4) 2024: 164 pages
  ✓ Atlas Copco A (S6) 2020: 144 pages
  ✓ Atlas Copco A (S6) 2019: 150 pages
  ✓ Atlas Copco A (S6) 2021: 148 pages
  ✓ Atlas Copco A (S6) 2022: 152 pages
  ✓ Atlas Copco A (S6) 2024: 169 pages
  ✓ Atlas Copco A (S6) 2023: 154 pages
  ✓ B

In [ ]:
# Validate successful downloads, copy to companies_validated/{CID}/, and update coverage table
import importlib.util
from datetime import datetime

# Load validation function from scripts
validate_script_path = repo / 'scripts' / 'validate_reports.py'
spec = importlib.util.spec_from_file_location("validate_reports", validate_script_path)
validate_module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(validate_module)
validate_pdf = validate_module.validate_pdf

# Setup directories
validated_dir = repo / 'companies_validated'
validated_dir.mkdir(exist_ok=True)

# Reload coverage table (in case it was modified)
coverage_df = pd.read_csv(coverage_path, sep='\t')

validation_records = []

for _, dl in successes.iterrows():
    cid = dl['cid']
    company = dl['company']
    year = dl['year']
    output_path = dl.get('output_path', '')
    
    if not output_path:
        output_path = str(repo / 'companies' / cid / f"{cid}_{year}_Annual_Report.pdf")
    
    pdf_path = Path(output_path)
    
    if not pdf_path.exists():
        validation_records.append({
            'company': company, 'cid': cid, 'year': year,
            'valid': False, 'issues': 'file missing after download'
        })
        print(f"⚠ Missing file: {pdf_path}")
        continue

    # Run validation
    validation = validate_pdf(str(pdf_path), company_name=company, expected_year=year)
    is_valid = validation.get('valid', False)
    issues = '; '.join(validation.get('issues', [])) if validation.get('issues') else ''
    size_mb = os.path.getsize(pdf_path) / (1024 * 1024)
    
    validation_records.append({
        'company': company, 'cid': cid, 'year': year,
        'valid': is_valid, 'issues': issues,
        'confidence': validation.get('confidence'),
        'pages': validation.get('pages'),
        'size_mb': size_mb
    })

    if is_valid:
        # Copy to validated directory
        dest_dir = validated_dir / cid
        dest_dir.mkdir(exist_ok=True)
        dest_path = dest_dir / pdf_path.name
        dest_path.write_bytes(pdf_path.read_bytes())
        print(f"✓ Validated: {company} {year} ({validation.get('pages')} pages, {validation.get('confidence'):.1f}% confidence)")

        # Update coverage table
        mask = (coverage_df['Company_Identifier'] == cid) & (coverage_df['FiscalYear'] == year)
        coverage_df.loc[mask, 'Report_URL'] = dl.get('url', '')
        coverage_df.loc[mask, 'Source_Page'] = dl.get('source_page', '')
        coverage_df.loc[mask, 'Pages'] = validation.get('pages', '')
        coverage_df.loc[mask, 'Size_MB'] = f"{size_mb:.2f}"
        coverage_df.loc[mask, 'Validation_Status'] = 'Valid'
        coverage_df.loc[mask, 'Validation_Confidence'] = f"{validation.get('confidence', 0.0):.1f}%"
        coverage_df.loc[mask, 'Validation_Issues'] = issues
        coverage_df.loc[mask, 'Validation_Date'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        coverage_df.loc[mask, 'Priority'] = 'Complete ✓'
        coverage_df.loc[mask, 'CaptureStatus'] = 'Validated'
    else:
        print(f"✗ Failed validation: {company} {year} ({issues})")

# Summary
print('\n' + '=' * 50)
print('Validation Summary')
print('=' * 50)
valid_count = sum(1 for r in validation_records if r['valid'])
print(f"✓ Valid: {valid_count} / {len(validation_records)}")
print(f"✗ Invalid: {len(validation_records) - valid_count}")

if any(not r['valid'] for r in validation_records):
    print('\nInvalid reports:')
    for r in validation_records:
        if not r['valid']:
            print(f"  ✗ {r['company']} ({r['cid']}) {r['year']}: {r.get('issues','')}")

# Save updated coverage table
coverage_df.to_csv(coverage_path, sep='\t', index=False)
print(f"\n✓ Coverage table updated: {coverage_path}")

✓ Validated: Addtech B 2019 (61 pages, 62.9% confidence)


/var/folders/t2/wr5tn7h97pdb6tkkgvr3xqf00000gn/T/ipykernel_78465/1967899169.py:64: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'https://www.addtech.com/fileadmin/user_upload/Addtech-Annual-Report-2019-2020.pdf' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  coverage_df.loc[mask, 'Report_URL'] = dl.get('url', '')
/var/folders/t2/wr5tn7h97pdb6tkkgvr3xqf00000gn/T/ipykernel_78465/1967899169.py:65: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'https://www.addtech.com/investors-and-media/annual-reports' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  coverage_df.loc[mask, 'Source_Page'] = dl.get('source_page', '')
/var/folders/t2/wr5tn7h97pdb6tkkgvr3xqf00000gn/T/ipykernel_78465/1967899169.py:67: FutureWarning: Setting an item of incompatible 

✓ Validated: Addtech B 2020 (74 pages, 66.4% confidence)
✓ Validated: Addtech B 2021 (74 pages, 66.4% confidence)
✓ Validated: Addtech B 2022 (80 pages, 68.0% confidence)
✓ Validated: Addtech B 2024 (152 pages, 87.2% confidence)
✓ Validated: Addtech B 2023 (155 pages, 88.0% confidence)
✓ Validated: Alfa Laval 2019 (152 pages, 87.2% confidence)
✓ Validated: Alfa Laval 2020 (160 pages, 89.3% confidence)
✓ Validated: Alfa Laval 2021 (194 pages, 98.4% confidence)
✓ Validated: Alfa Laval 2023 (192 pages, 97.9% confidence)
✓ Validated: Alfa Laval 2024 (81 pages, 68.3% confidence)
✓ Validated: Alfa Laval 2022 (200 pages, 100.0% confidence)
✓ Validated: ASSA ABLOY B 2019 (108 pages, 75.5% confidence)
✓ Validated: ASSA ABLOY B 2020 (112 pages, 76.5% confidence)
✓ Validated: ASSA ABLOY B 2022 (116 pages, 77.6% confidence)
✓ Validated: ASSA ABLOY B 2021 (116 pages, 77.6% confidence)
✓ Validated: ASSA ABLOY B 2023 (116 pages, 77.6% confidence)
✓ Validated: ASSA ABLOY B 2024 (164 pages, 90.4% confi

ignore '/Perms' verify failed
ignore '/Perms' verify failed


✗ Failed validation: Nordea Bank Abp 2023 (Company name "Nordea Bank Abp" not found in PDF)
✗ Failed validation: Nordea Bank Abp 2020 (Company name "Nordea Bank Abp" not found in PDF)
✓ Validated: SAAB B 2020 (166 pages, 90.9% confidence)
✓ Validated: SAAB B 2019 (152 pages, 87.2% confidence)
✗ Failed validation: Nordea Bank Abp 2024 (Company name "Nordea Bank Abp" not found in PDF)
✓ Validated: SAAB B 2022 (172 pages, 92.5% confidence)
✓ Validated: SAAB B 2024 (228 pages, 100.0% confidence)
✓ Validated: SAAB B 2023 (180 pages, 94.7% confidence)
✓ Validated: SAAB B 2021 (168 pages, 91.5% confidence)
✓ Validated: Sandvik 2020 (148 pages, 86.1% confidence)
✓ Validated: Sandvik 2021 (160 pages, 89.3% confidence)
✓ Validated: Sandvik 2023 (168 pages, 91.5% confidence)
✓ Validated: Sandvik 2022 (160 pages, 89.3% confidence)
✓ Validated: Sandvik 2024 (160 pages, 89.3% confidence)
✓ Validated: SCA B 2019 (168 pages, 91.5% confidence)
✓ Validated: SCA B 2020 (180 pages, 94.7% confidence)
✓ Val

In [36]:
# Append new validation records to validation_results.csv
if validation_records:
    new_records = []
    for r in validation_records:
        if r['valid']:
            cid = r['cid']
            year = r['year']
            company = r['company']
            pdf_name = f"{cid}_{year}_Annual_Report.pdf"
            
            # Find the download result for this record
            dl_info = successes[(successes['cid'] == cid) & (successes['year'] == year)].iloc[0] if len(successes[(successes['cid'] == cid) & (successes['year'] == year)]) > 0 else {}
            
            new_records.append({
                'CID': cid,
                'Company_Name': company,
                'Year': year,
                'Filename': pdf_name,
                'Valid': True,
                'Confidence': r.get('confidence', 0),
                'Pages': r.get('pages', 0),
                'Company_Found': True,
                'Year_Found': True,
                'Issues': r.get('issues', ''),
                'Source_Path': f"companies/{cid}/{pdf_name}",
                'Validated_Date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                'IR_URL': dl_info.get('source_page', '') if isinstance(dl_info, dict) else (dl_info['source_page'] if 'source_page' in dl_info.index else ''),
                'Pattern_Type': 'Downloaded via notebook'
            })
    
    if new_records:
        # Load existing validation results
        existing_validation = pd.read_csv(validation_path)
        new_df = pd.DataFrame(new_records)
        
        # Remove duplicates (in case we're re-running)
        existing_keys = set(zip(existing_validation['CID'], existing_validation['Year']))
        new_df = new_df[~new_df.apply(lambda r: (r['CID'], r['Year']) in existing_keys, axis=1)]
        
        if len(new_df) > 0:
            updated_validation = pd.concat([existing_validation, new_df], ignore_index=True)
            updated_validation.to_csv(validation_path, index=False)
            print(f"✓ Added {len(new_df)} new records to validation_results.csv")
        else:
            print("ℹ All validated records already exist in validation_results.csv")
else:
    print("ℹ No validation records to add")

✓ Added 119 new records to validation_results.csv


/var/folders/t2/wr5tn7h97pdb6tkkgvr3xqf00000gn/T/ipykernel_78465/1188981406.py:41: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  updated_validation = pd.concat([existing_validation, new_df], ignore_index=True)
